Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [x] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [x] Engineer at least two new features. (See below for explanation & ideas.)
- [x] Fit a linear regression model with at least two features.
- [x] Get the model's coefficients and intercept.
- [x] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [x] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [x] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [20]:
# adding month to be able to easier filter into test/train
def add_month(cell):
  if cell[5:7] == '04':
    return 'april'
  elif cell[5:7] == '05':
    return 'may'
  else:
    return 'june'

df['month'] = df['created'].apply(add_month)
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,month
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,june
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,june
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,april
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,april
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,april


In [25]:
# engineered feature 1:

perk_list = ["elevator", "cats_allowed", "hardwood_floors", "dogs_allowed", 
             "doorman", "dishwasher",	"no_fee",	"laundry_in_building",	
             "fitness_center",	"pre-war",	"laundry_in_unit",	"roof_deck",
             "outdoor_space",	"dining_room",	"high_speed_internet",	
             "balcony",	"swimming_pool",	"new_construction",	"terrace",	
             "exclusive",	"loft",	"garden_patio",	"wheelchair_access",	
             "common_outdoor_space"]

def get_total_perks(cell):
  result = 0
  for perk in perk_list:
    if cell[perk]:
      result += 1
  return result

df['total_perks'] = df.apply(get_total_perks, axis=1)
df['total_perks'].head()

0    0
1    5
2    3
3    2
4    1
Name: total_perks, dtype: int64

In [26]:
# what is the average num of perks for an apartment?
df['total_perks'].mean()

4.672593563717557

In [31]:
# engineered feature 2:
import numpy as np 

def get_description_length(description):
  if isinstance(description, (float)):
    return 0
  return len(description)

df['description_length'] = df['description'].apply(get_description_length)
df['description_length'].mean()

601.8062150480365

In [34]:
# splitting into train and test
april = df['month'] == 'april'
may = df['month'] == 'may'
june = df['month'] == 'june'

train = df[may | april]
print("Train shape:", train.shape)
test = df[june]
print("Test shape:", test.shape)

Train shape: (31844, 37)
Test shape: (16973, 37)


In [35]:
# fitting model to my 2 features and bedrooms

features = ['bedrooms', 'total_perks', 'description_length']
target = 'price'

X_train = train[features]
X_test = test[features]

y_train = train[target]
y_test = test[target]

print("Shape of train:", X_train.shape)
print("Shape of test:", X_test.shape)

Shape of train: (31844, 3)
Shape of test: (16973, 3)


In [0]:
# instantiating the model 
from sklearn.linear_model import LinearRegression

model = LinearRegression()

In [0]:
# making function for getting: rmse, mae, and r^2
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

def get_metrics(y_pred, y_real):
  print("RMSE:", np.sqrt(mean_squared_error(y_pred, y_real)))
  print("MAE:", mean_absolute_error(y_pred, y_real))
  print("R^2:", r2_score(y_pred, y_real))

In [38]:
# fitting the model and checking metrics
model.fit(X_train, y_train)
y_pred = model.predict(X_train)
get_metrics(y_pred, y_train)

RMSE: 1424.4964007319447
MAE: 924.410146634132
R^2: -0.8861478927212534


In [39]:
# checking metrics on test data
y_pred = model.predict(X_test)
get_metrics(y_pred, y_test)

RMSE: 1425.0310303603706
MAE: 934.5360115295557
R^2: -0.8905998893363913


In [40]:
def get_model_equation(model):
  print(f"Intercept: {model.intercept_}")
  print(f"Coefficients: {model.coef_}")

get_model_equation(model)

Intercept: 1740.6213201308194
Coefficients: [8.10354545e+02 1.23783200e+02 2.22021916e-02]


In [0]:
# training model 2 on new features:

features = ["bedrooms", "bathrooms", "elevator"]
target = "price"

X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

In [0]:
model = LinearRegression()

In [43]:
model.fit(X_train, y_train)
y_pred = model.predict(X_train)
get_metrics(y_pred, y_train)

RMSE: 1206.1364913647317
MAE: 791.0547045751715
R^2: 0.11846460134520354


In [44]:
y_pred = model.predict(X_test)
get_metrics(y_pred, y_test)

RMSE: 1195.4519659862676
MAE: 798.8568459132107
R^2: 0.12523808324333718


In [45]:
# final model, first will add rooms as a feature

def add_rooms(cell):
  return cell['bedrooms'] + cell['bathrooms']

df['rooms'] = df.apply(add_rooms, axis=1)
df['rooms'].head()

0    4.5
1    3.0
2    2.0
3    2.0
4    5.0
Name: rooms, dtype: float64

In [47]:
# re splitting since I added a new column
april = df['month'] == 'april'
may = df['month'] == 'may'
june = df['month'] == 'june'

train = df[may | april]
print("Train shape:", train.shape)
test = df[june]
print("Test shape:", test.shape)

Train shape: (31844, 38)
Test shape: (16973, 38)


In [0]:
features = ['rooms', 'bedrooms', 'longitude']
target = 'price'

X_train = train[features]
X_test = test[features]
y_train = train[target]
y_test = test[target]

In [0]:
model = LinearRegression()

In [75]:
model.fit(X_train, y_train)
y_pred = model.predict(X_train)
get_metrics(y_pred, y_train)

RMSE: 1148.1865803426144
MAE: 737.4172130391559
R^2: 0.26213940433836025


In [76]:
y_pred = model.predict(X_test)
get_metrics(y_pred, y_test)

RMSE: 1134.1908185234622
MAE: 743.52524347017
R^2: 0.2691629663132806
